# Multimodal Model in Snowflake

## Imports

In [ ]:
from snowflake.snowpark.functions import call_udf, call_builtin, col, lit, object_construct, regexp_replace

# Import python packages
import streamlit as st
import pandas as pd
from io import BytesIO
import base64

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Example: Describe a list of Images

## Retrieve Data

In [ ]:
files_df = session.sql('SELECT * FROM DIRECTORY(@IMAGE_FILES)').filter(col('RELATIVE_PATH').startswith('celebs/')).limit(2).cache_result()
files_df

## Define Prompt

In [ ]:
# Define how to retrieve the presigned URL
file_url = call_builtin('GET_PRESIGNED_URL', lit('@IMAGE_FILES'), col('RELATIVE_PATH'))

# Define vLLM generation arguments
generation_args = object_construct(
    lit('max_length'),lit(2500),
    lit('top_p'),lit(0.8)
)

# Define vLLM arguments
args = object_construct(
    lit('file_url'),file_url,
    lit('stream'),lit(False),
    lit('generation_args'),generation_args
)

# Define vLLM prompt
prompt = object_construct(
    lit('prompt'),lit('Who is this?'),
    lit('args'),args
)

model = call_udf('GLM_V4_9B',prompt)

## Apply Model to Data

In [ ]:
analysis_df = files_df.with_column('VLLM_OUTPUT', model)
analysis_df = analysis_df.with_column('PRESIGNED_URL', file_url)
analysis_df = analysis_df.with_column('LLM_OUTPUT_TEXT', col('VLLM_OUTPUT')['LLM_OUTPUT_TEXT'].cast('string')).cache_result()
analysis_df[['RELATIVE_PATH','LLM_OUTPUT_TEXT']]

## Visualize Results

In [ ]:
for i, row in analysis_df.to_pandas().iterrows():
    with st.container():
        col1, col2 = st.columns([0.2,0.8])
        with col1:
            st.image(row['PRESIGNED_URL'], caption=row['RELATIVE_PATH'])
        with col2:
            st.markdown(row['LLM_OUTPUT_TEXT'])
        st.markdown("---")

# Example: Score Multipage PDFs

## Retrieve Data

In [ ]:
files_df = session.sql('SELECT * FROM DIRECTORY(@IMAGE_FILES)').filter(col('RELATIVE_PATH').startswith('pdfs/')).limit(2).cache_result()
# Define which pages of the PDF you want to score
from_page = 18
to_page = 26
files_df = files_df.cross_join(session.range(from_page, to_page, 1))
files_df

## Define Prompt

In [ ]:
# Define how to retrieve the presigned URL
file_url = call_builtin('GET_PRESIGNED_URL', lit('@IMAGE_FILES'), col('RELATIVE_PATH'))

# Define vLLM generation arguments
generation_args = object_construct(
    lit('max_length'),lit(2500),
    lit('top_p'),lit(0.8)
)

# Define vLLM arguments
args = object_construct(
    lit('file_url'),file_url,
    lit('stream'),lit(False),
    lit('pdf_page'),col('ID'),
    lit('return_image_base64'),lit(True), # make sure to return pdf pages as base64 images so we can visualize them in this notebooks
    lit('generation_args'),generation_args
)

# Define vLLM prompt
prompt = object_construct(
    lit('prompt'),lit('Explain the graphs in this slide to me in maximum 5 sentences.'),
    lit('args'),args
)

model = call_udf('GLM_V4_9B',prompt)

## Apply Model to Data

In [ ]:
analysis_df = files_df.with_column('VLLM_OUTPUT', model).cache_result()
analysis_df = analysis_df.with_column('PRESIGNED_URL', file_url)
analysis_df = analysis_df.with_column('LLM_OUTPUT_TEXT', col('VLLM_OUTPUT')['LLM_OUTPUT_TEXT'].cast('string'))
analysis_df = analysis_df.with_column('BASE64_IMAGE', col('VLLM_OUTPUT')['base64_image'].cast('string')).cache_result()
analysis_df = analysis_df.order_by('ID')
analysis_df

## Visualize Results

In [ ]:
from PIL import Image
import base64
from io import BytesIO
def base64_to_pil_image(base64_str: str) -> Image.Image:
    # Decode the base64 string to bytes
    img_byte_data = base64.b64decode(base64_str)
    # Create a BytesIO buffer from the byte data
    img_buffer = BytesIO(img_byte_data)
    # Open the image from the buffer using PIL
    image = Image.open(img_buffer)
    return image
    
for i, row in analysis_df.to_pandas().iterrows():
    with st.container():
        col1, col2 = st.columns([0.5,0.5])
        with col1:
            image = base64_to_pil_image(row['BASE64_IMAGE'])
            st.image(image, caption=f"{row['RELATIVE_PATH']} - Page: {row['ID']}")
        with col2:
            st.markdown(row['LLM_OUTPUT_TEXT'])
        st.markdown("---")

# Example: Extracting Attributes from Images

### Retrieve Data

In [ ]:
files_df = session.sql('SELECT * FROM DIRECTORY(@IMAGE_FILES)').filter(col('RELATIVE_PATH').startswith('celebs/')).limit(2).cache_result()
files_df

### Define Prompt

In [ ]:
# Define how to retrieve the presigned URL
file_url = call_builtin('GET_PRESIGNED_URL', lit('@IMAGE_FILES'), col('RELATIVE_PATH'))

# Define vLLM generation arguments
generation_args = object_construct(
    lit('max_length'),lit(2500),
    lit('top_p'),lit(0.8)
)

# Define vLLM arguments
args = object_construct(
    lit('file_url'),file_url,
    lit('stream'),lit(False),
    lit('generation_args'),generation_args
)

# Define vLLM prompt
prompt = object_construct(
    lit('prompt'),lit('Given this picture, return a JSON like this: {haircolor:haircolor, gender:gender, approx_age:approx_age}.'),
    lit('args'),args
)

model = call_udf('GLM_V4_9B',prompt)

## Apply Model to Data

In [ ]:
analysis_df = files_df.with_column('VLLM_OUTPUT', model)
analysis_df = analysis_df.with_column('PRESIGNED_URL', file_url)
analysis_df = analysis_df.with_column('LLM_OUTPUT_TEXT', col('VLLM_OUTPUT')['LLM_OUTPUT_TEXT'])
analysis_df = analysis_df.with_column(
    'LLM_OUTPUT_JSON', 
    call_builtin('try_parse_json', regexp_replace("LLM_OUTPUT_TEXT", r"(^[^{}]*|[^{}]*$)", '', 1, 0, 's'))).cache_result()
analysis_df[['RELATIVE_PATH','LLM_OUTPUT_JSON']]

## Visualize Results

In [ ]:
for i, row in analysis_df.to_pandas().iterrows():
    with st.container():
        col1, col2 = st.columns([0.2,0.8])
        with col1:
            st.image(row['PRESIGNED_URL'], caption=row['FILE_URL'], use_column_width=True)
        with col2:
            st.json(row['LLM_OUTPUT_JSON'])
        st.markdown("---")

# Example: Explaining Graphs

## Generate a graph and save figure as base64 image

In [ ]:
import matplotlib.pyplot as plt

# Data for the revenue by month in 2024
months = [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]
revenue = [1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500]

# Create a bar plot
plt.figure(figsize=(5, 3))
plt.bar(months, revenue, color='blue')

# Add titles and labels
plt.title("Revenue by Month for the Year 2024")
plt.xlabel("Month")
plt.ylabel("Revenue (in USD)")
plt.xticks(rotation=45)
plt.ylim(0, 7000)

# Save the figure to a BytesIO object
buf = BytesIO()
plt.savefig(buf, format='png')
buf.seek(0)

# Convert the BytesIO object to a base64 string
img_base64 = base64.b64encode(buf.read()).decode('utf-8')

# Close the buffer
buf.close()

## Apply Model to Data

In [ ]:
# Define vLLM generation arguments
generation_args = object_construct(
    lit('max_length'),lit(2500),
    lit('top_p'),lit(0.8)
)

# Define vLLM arguments
args = object_construct(
    lit('base64_image_string'),lit(img_base64),
    lit('stream'),lit(False),
    lit('generation_args'),generation_args
)

# Define vLLM prompt
prompt = object_construct(
    lit('prompt'),lit('How much did the revenue grew from January to December in 2024?'),
    lit('args'),args
)

model = call_udf('GLM_V4_9B',prompt)

In [ ]:
plot_explanation_df = session.range(1).with_column('VLLM_OUTPUT', model).cache_result()
plot_explanation_df = plot_explanation_df.with_column('LLM_OUTPUT_TEXT', col('VLLM_OUTPUT')['LLM_OUTPUT_TEXT'].cast('string')).cache_result()
plot_explanation_df

In [ ]:
st.info(plot_explanation_df[['LLM_OUTPUT_TEXT']].collect()[0]['LLM_OUTPUT_TEXT'])